# Pendulum dynamics

by Thomas Mohren
2019-04-26


The equations of motion for the pendulum of arbitrary shape can be derived using Lagrange's equation, leading to: 

$\ddot{\theta} = \frac{1}{I} \left( (\sum_{i=1}^n m_i L_{i,cg} ) g  \sin \theta - b \dot{\theta}  \right) $, 

where $\theta$ is the angle of the pendulum (0 is downwards), 

$I$ is the rotational inertia of the pendulum, 

$m_i$ is the mass, 

$L_{i,cg}$ is the length from the base to the center of gravity for the $i^{th}$ component, 

$g$ is gravity (note that in this formulation, gravity is negative because I defined y to be positive upwards), 

and $b$ is damping. 

I consider the case where the pendulum has a leg and bulb that both have mass. $I$ is then given by $m_\text{bulb} L_\text{bulb}^2 + \frac{ m_\text{Leg} L_\text{Leg}^2}{3} $.


## Swing period of the linearized pendulum 
The pendulum has two fixed points about we can linearize. 

When we linearize about the downward fixed point we can define a characteristic swinging period: 

$ T = 2 \pi \sqrt{ \frac{L}{g} } $.  

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy.integrate import odeint  
from IPython.display import HTML
from matplotlib import animation, rc 

In [2]:
# define pendulum dynamics function 
def pendulum_dynamics( y ,t, b=0.001, L=1, m_b=1, m_L=1, g=-10 ):
    I = m_b*L**2 + 1/3*m_L*L**2
    dydt = np.zeros(2) 
    dydt[0] = y[1]
    dydt[1] = 1/I * (  (L*m_b+L/2*m_L)*g*np.sin(y[0]) - b*y[1]  )
    return dydt 

# function to wrap theta 
def wrap2periodic(theta, period = 2*np.pi, center = 0 ):
    thetaWrapped = (theta -np.pi+ center) % period -np.pi+ center 
    return thetaWrapped

# function for having multiples of pi on the x-axis in plot 
def multiple_formatter(denominator=2, number=np.pi, latex='\pi'):
    def gcd(a, b):
        while b:
            a, b = b, a%b
        return a
    def _multiple_formatter(x, pos):
        den = denominator
        num = np.int(np.rint(den*x/number))
        com = gcd(num,den)
        (num,den) = (int(num/com),int(den/com))
        if den==1:
            if num==0:
                return r'$0$'
            if num==1:
                return r'$%s$'%latex
            elif num==-1:
                return r'$-%s$'%latex
            else:
                return r'$%s%s$'%(num,latex)
        else:
            if num==1:
                return r'$\frac{%s}{%s}$'%(latex,den)
            elif num==-1:
                return r'$\frac{-%s}{%s}$'%(latex,den)
            else:
                return r'$\frac{%s%s}{%s}$'%(num,latex,den)
    return _multiple_formatter

In [3]:
# parameters
L = 1.  # pendulum length
m_L = 1. # total leg mass 
m_b = 1. # total bulb mass 
b = 0.3 # damping parameter 
g = -10 # gravity 
y0 = [0.3,8] # set initial conditions  

L_eff = (m_b*L + m_L*L/2)/(m_L+m_b)
T = 2*np.pi * np.sqrt( L_eff/np.abs(g) ) 
# (3) print 'First number %d and second number is %d' % (first, second)
print('Swing period is %f  sec.'%(T) )

#time parameters for main simulation 
dt = 0.01; 
tLast = 10 ; 
tInt = np.arange(0,tLast+dt, dt) ;

# plotting parameters
figCenter = 0 # center around fixed point 0 , or np.pi 
n_frames = int( tLast/dt ) # time parameters 
fps = 100  # animation parameters 
window = 1.1  # sets window size 
dot_scale = 500  # size of pendulum bulb 
window = L*1.2 # set window size as multiple of pendulum length
w_2 = m_L/2*0.05 # determine width of pendulum stalk

# y = odeint(pendulum_dynamics, y0, tInt )  
y = odeint(pendulum_dynamics, y0, tInt, args=( b,L,m_b,m_L,g ) )   

Swing period is 1.720721  sec.


In [10]:
# for animation: initialization function to plot the background of each frame
def init():  
    line.set_data([], [])
    dots.set_offsets( []  )
    scat.set_offsets( []  )
    scatmany.set_offsets([] )
    return (line,dots,scat,scatmany,rectangle )

# # for animation: update line and scatter plots
def animate(i):  
    x_pos = np.array( np.hstack([0, np.multiply( L ,np.sin(y[i,:1]) ) ]) )
    y_pos = np.array( np.hstack([0, np.multiply( L, -np.cos(y[i,:1])  ) ]))

    th = wrap2periodic( y[i,0],2*np.pi, figCenter)
    dth = y[i,1] 
    
    line.set_data( x_pos,y_pos ) 
    dots.set_offsets( np.vstack( (x_pos,y_pos )  ).transpose().tolist()  )  
    scat.set_offsets( np.vstack( (th,dth )  ).transpose().tolist()  )      
    
    if i<2: 
        thV = [0,0.1] 
        dthV = [0,0.1]
    else: 
        thV = wrap2periodic( y[ :i,0],2*np.pi, figCenter)
        dthV = y[ :i,1]
    scatmany.set_offsets(   np.vstack( (thV,dthV )  ).transpose().tolist()    ) 
    
    rectangle.angle = th*180/np.pi
    rectangle.xy = ( - np.cos(th)*w_2, -np.sin(th)*w_2 ) 
    
    return (line,dots,scat,scatmany,rectangle) 

fig,axs = plt.subplots(1,2,figsize=(12,6)) 
%matplotlib inline 

m = np.array( [0.1,m_b] )
 
x_pos = np.array( np.hstack([0, np.multiply( L , np.sin(y0[:1]))    ]) )
y_pos = np.array( np.hstack([0, np.multiply( L, -np.cos(y0[:1])) ]))

th = wrap2periodic( y[0,0],2*np.pi, figCenter )
dth = y[0,1]

baseline = axs[0].plot([-1,1],[0,0],'k', zorder = -3) 
line, = axs[0].plot( x_pos, y_pos ) 
dots  = axs[0].scatter( x_pos ,y_pos,s=m*dot_scale )
scat = axs[1].scatter( th, dth ) 
scatmany = axs[1].scatter(  th, dth ,s=1, color='black',zorder = -1  ) 
rectangle = plt.Rectangle((-w_2 ,0), 2*w_2, -L,angle = th*180/np.pi,zorder=-1)
axs[0].add_patch(rectangle)

# set plot parameters 
axs[0].set_xlim( ( np.array([-1.,1.])*window  ))
axs[0].set_ylim( ( np.array([-1.,1.])*window ))
axs[0].set_aspect('equal')
axs[0].tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    left=False,      # ticks along the left edge are off
    right=False,         # ticks along the right edge are off
    labelbottom=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off
axs[1].set_xlim( ( np.array([-np.pi,np.pi])   ))
axs[1].set_ylim( ( np.array([-1.,1.])*10 ))
axs[1].xaxis.set_major_locator(plt.MultipleLocator(np.pi )) 
axs[1].xaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter())) 
axs[1].set_xlabel(r'$\theta$' "\n" 'angular position (rad)')
axs[1].set_ylabel(r'$\dot{\theta}$' "\n" '\n angular velocity (rad/s)'  )
axs[1].yaxis.set_major_locator(plt.MaxNLocator(3))
axs[1].set_title('Phase plane of the pendulum')
fig.tight_layout()

#  call the animator.
anim = animation.FuncAnimation(fig, animate , init_func=init,
                               frames=n_frames ,interval= int( 1000/fps ), blit=True)
HTML(anim.to_html5_video())

In [ ]:
# save the animation
Writer = animation.writers['ffmpeg']
writer = Writer(fps=fps, metadata=dict(artist='Me'), bitrate=1800)
anim.save('pendulum_animation.mp4', writer=writer)